In [2]:
import os
import glob
import librosa
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib

# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def extract_mfcc_features(audio_path, n_mfcc=25, n_fft=2048, hop_length=512):
    try:
        audio_data, sr = librosa.load(audio_path, sr=None)
    except Exception as e:
        print(f"Error loading audio file {audio_path}: {e}")
        return None

    mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    delta_mfccs = librosa.feature.delta(mfccs)
    delta_delta_mfccs = librosa.feature.delta(delta_mfccs)
    combined_features = np.vstack([mfccs, delta_mfccs, delta_delta_mfccs])

    return np.mean(combined_features.T, axis=0)

def create_dataset(directory, label):
    X, y = [], []
    audio_files = glob.glob(os.path.join(directory, "*.wav"))
    for audio_path in audio_files:
        mfcc_features = extract_mfcc_features(audio_path)
        if mfcc_features is not None:
            X.append(mfcc_features)
            y.append(label)
        else:
            print(f"Skipping audio file {audio_path}")

    print("Number of samples in", directory, ":", len(X))
    print("Filenames in", directory, ":", [os.path.basename(path) for path in audio_files])
    return X, y

def train_model(X, y):
    unique_classes = np.unique(y)
    print("Unique classes in y_train:", unique_classes)

    if len(unique_classes) < 2:
        raise ValueError("At least 2 classes are required to train")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Expanded Grid Search for hyperparameter tuning
    param_grid = {
        'C': [0.1, 1, 10, 100, 1000],
        'kernel': ['rbf'],
        'gamma': ['scale', 'auto', 0.01, 0.001, 0.0001]
    }
    grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5, verbose=3)
    grid_search.fit(X_train_scaled, y_train)

    # Get the best hyperparameters
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)

    svm_classifier = SVC(**best_params, random_state=42)
    svm_classifier.fit(X_train_scaled, y_train)

    y_pred = svm_classifier.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    confusion_mtx = confusion_matrix(y_test, y_pred)

    print("Metrics on Test Set:")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Confusion Matrix:")
    print(confusion_mtx)

    # Save the trained SVM model and scaler
    model_filename = "svm_model.pkl"
    scaler_filename = "scaler.pkl"
    joblib.dump(svm_classifier, model_filename)
    joblib.dump(scaler, scaler_filename)

def analyze_audio(input_audio_path):
    model_filename = "svm_model.pkl"
    scaler_filename = "scaler.pkl"
    svm_classifier = joblib.load(model_filename)
    scaler = joblib.load(scaler_filename)

    if not os.path.exists(input_audio_path):
        print("Error: The specified file does not exist.")
        return
    elif not input_audio_path.lower().endswith(".wav"):
        print("Error: The specified file is not a .wav file.")
        return

    mfcc_features = extract_mfcc_features(input_audio_path)

    if mfcc_features is not None:
        mfcc_features_scaled = scaler.transform(mfcc_features.reshape(1, -1))
        prediction = svm_classifier.predict(mfcc_features_scaled)
        if prediction[0] == 0:
            print("The input audio is classified as genuine.")
        else:
            print("The input audio is classified as deepfake.")
    else:
        print("Error: Unable to process the input audio.")

def main():
    genuine_dir = r"/AUDIO/real_audio"
    deepfake_dir = r"/AUDIO/deepfake_audio"

    X_genuine, y_genuine = create_dataset(genuine_dir, label=0)
    X_deepfake, y_deepfake = create_dataset(deepfake_dir, label=1)

    # Check if each class has at least two samples
    if len(X_genuine) < 2 or len(X_deepfake) < 2:
        print("Each class should have at least two samples for stratified splitting.")
        print("Combining both classes into one for training.")
        X = np.vstack((X_genuine, X_deepfake))
        y = np.hstack((y_genuine, y_deepfake))
    else:
        X = np.vstack((X_genuine, X_deepfake))
        y = np.hstack((y_genuine, y_deepfake))

    train_model(X, y)

if __name__ == "__main__":
    main()

    user_input_file = input("Enter the path of the .wav file to analyze: ")
    analyze_audio(user_input_file)


Number of samples in real_audio : 10004
Filenames in real_audio : ['Furqanreal.wav', 'HimanshuReal.wav', 'LJ001-0001.wav', 'LJ001-0002.wav', 'LJ001-0003.wav', 'LJ001-0004.wav', 'LJ001-0005.wav', 'LJ001-0006.wav', 'LJ001-0007.wav', 'LJ001-0008.wav', 'LJ001-0009.wav', 'LJ001-0010.wav', 'LJ001-0011.wav', 'LJ001-0012.wav', 'LJ001-0013.wav', 'LJ001-0014.wav', 'LJ001-0015.wav', 'LJ001-0016.wav', 'LJ001-0017.wav', 'LJ001-0018.wav', 'LJ001-0019.wav', 'LJ001-0020.wav', 'LJ001-0021.wav', 'LJ001-0022.wav', 'LJ001-0023.wav', 'LJ001-0024.wav', 'LJ001-0025.wav', 'LJ001-0026.wav', 'LJ001-0027.wav', 'LJ001-0028.wav', 'LJ001-0029.wav', 'LJ001-0030.wav', 'LJ001-0031.wav', 'LJ001-0032.wav', 'LJ001-0033.wav', 'LJ001-0034.wav', 'LJ001-0035.wav', 'LJ001-0036.wav', 'LJ001-0037.wav', 'LJ001-0038.wav', 'LJ001-0039.wav', 'LJ001-0040.wav', 'LJ001-0041.wav', 'LJ001-0042.wav', 'LJ001-0043.wav', 'LJ001-0044.wav', 'LJ001-0045.wav', 'LJ001-0046.wav', 'LJ001-0047.wav', 'LJ001-0048.wav', 'LJ001-0049.wav', 'LJ001-0050.w